<a href="https://colab.research.google.com/github/ehBeak/animationProject/blob/main/RVM_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Robust High-Resolution Video Matting with Temporal Guidance.

![Teaser](https://raw.githubusercontent.com/PeterL1n/RobustVideoMatting/master/documentation/image/teaser.gif)

[Project Site](https://peterl1n.github.io/RobustVideoMatting) | [GitHub](https://github.com/PeterL1n/RobustVideoMatting) | [Paper](https://arxiv.org/abs/2108.11515)


## Colab Demo
This colab notebook is set up to let you quickly test our model on your video.

### Option 1: Upload your video

Run the cell below and upload your own video. (Only tested on .mp4 files)

In [ ]:
import os
from google.colab import files

uploaded = files.upload()           # Use colab upload dialog.
uploaded = list(uploaded.keys())    # Get uploaded filenames.
assert len(uploaded) == 1           # Make sure only uploaded one file.
os.rename(uploaded[0], 'input.mp4') # Rename file to "input.mp4".

### Option 2: Try our demo video

Run the cell below to download our demo video. Skip it if you have uploaded your own video.

In [ ]:
!gdown https://drive.google.com/uc?id=1I0v72-hNlK1hm9q1OwyaATUYApXpotS6 -O input.mp4

### Start Inference

Run the cells below to process your video.

In [ ]:
!pip install --quiet av pims

In [ ]:
import torch

model = torch.hub.load("PeterL1n/RobustVideoMatting", "mobilenetv3").cuda() # or "resnet50"
convert_video = torch.hub.load("PeterL1n/RobustVideoMatting", "converter")

In [ ]:
convert_video(
    model,                           # The loaded model, can be on any device (cpu or cuda).
    input_source='input.mp4',        # A video file or an image sequence directory.
    downsample_ratio=None,           # [Optional] If None, make downsampled max size be 512px.
    output_type='video',             # Choose "video" or "png_sequence"
    output_composition='com.mp4',    # File path if video; directory path if png sequence.
    output_alpha="pha.mp4",          # [Optional] Output the raw alpha prediction.
    output_foreground="fgr.mp4",     # [Optional] Output the raw foreground prediction.
    output_video_mbps=4,             # Output video mbps. Not needed for png sequence.
    seq_chunk=12,                    # Process n frames at once for better parallelism.
    num_workers=1,                   # Only for image sequence input. Reader threads.
    progress=True                    # Print conversion progress.
)

### Download results

After the processing is done, you can download the results from the files panel on the left.

* `com.mp4`: The matting result composited on top of a green screen background.
* `pha.mp4`: The raw alpha matte.
* `fgr.mp4`: The raw foreground prediction.

In [ ]:
# mask씌우기
import cv2 as cv
import numpy as np

vidcap = cv.VideoCapture('pha.mp4')
sucess, image = vidcap.read()

vidcapCom = cv.VideoCapture('com.mp4')
sucesscom, imagecom = vidcapCom.read()

count = 1
sucess = True

while sucess:
  sucess, image = vidcap.read()
  #######
  image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
  mask = cv.inRange(image,(0,0,210),(359,255,255))
  mask = cv.bitwise_not(mask)

  #######
  sucesscom, imagecom = vidcapCom.read()

  #masked = cv.bitwise_and(imagecom, image)
  ##
  imagecom = cv.cvtColor(imagecom, cv.COLOR_BGR2BGRA)
  imagecom[mask>0]=(0,0,0,0)
  ##
  cv.imwrite('/content/masked/%d.png' % count, imagecom)

  

  if cv.waitKey(0) == 27:
    break
  count += 1

In [ ]:
# 녹색배경 제거

import cv2 as cv
import numpy as np

vidcapCom = cv.VideoCapture('com.mp4')
sucesscom, origin = vidcapCom.read()

count = 1
sucess = True

while sucesscom:
  sucesscom, origin = vidcapCom.read()

  imagecom = origin

  imagecom = cv.cvtColor(imagecom, cv.COLOR_BGR2HSV)
  mask = cv.inRange(imagecom,(50,0,0),(80,255,255))
  mask = cv.GaussianBlur(mask,(3,3),3,3) 
  #mask = cv.bitwise_not(mask)
  
  origin = cv.cvtColor(origin, cv.COLOR_BGR2BGRA)
  origin[mask>0]=(0,0,0,0)



  cv.imwrite('/content/delete/%d.png' % count, origin)

  

  if cv.waitKey(0) == 27:
    break
  count += 1